In [1]:
import tensorflow as tf
import os

In [2]:
W = tf.Variable(tf.zeros([5,1]),name="weights")
b = tf.Variable(0.,name="bias")

In [3]:
def combine_inputs(X):
    return tf.matmul(X,W) + b;

In [4]:
def interference(X):
    return tf.sigmoid(combine_inputs(X))

In [5]:
def loss(X,Y):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=combine_inputs(X),logits=Y))

In [6]:
def read_csv(batch_size,file_name, record_defaults):
    filename_queue = tf.train.string_input_producer([os.path.dirname("__file__") + "/Users/suhyeongcho/" + file_name])
    reader = tf.TextLineReader(skip_header_lines = 1)
    key, value = reader.read(filename_queue)
    
    decoded = tf.decode_csv(value,record_defaults=record_defaults)
    
    return tf.train.shuffle_batch(decoded,batch_size=batch_size,capacity=batch_size*50,min_after_dequeue=batch_size)

In [7]:
def inputs():
    passenger_id,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked = read_csv(100,"train.csv",[[0.0],[0.0],[0],[""],[""],[0.0],[0.0],[0.0],[""],[0.0],[""],[""]])
    
    is_first_class = tf.to_float(tf.equal(pclass,[1]))
    is_second_class = tf.to_float(tf.equal(pclass,[2]))
    is_third_class = tf.to_float(tf.equal(pclass,[3]))
    
    gender = tf.to_float(tf.equal(sex,["female"]))
    
    features = tf.transpose(tf.stack([is_first_class,is_second_class,is_third_class,gender,age]))
    survived = tf.reshape(survived,[100,1])
    
    return features,survived

In [8]:
def train(total_loss):
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

In [9]:
def evaluate(sess,X,Y):
    predicted = tf.cast(interference(X) > 0.5, tf.float32)
    
    print(sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted,Y),tf.float32))))

In [15]:
with tf.Session() as sess:
    
    tf.global_variables_initializer().run()
    
    X, Y = inputs()
    
    total_loss = loss(X,Y)
    train_op = train(total_loss)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess,coord=coord)
    
    training_steps = 1000
    
    for step in range(training_steps):
        sess.run([train_op])
        if step % 10 == 0:
            print("loss : ", sess.run([total_loss]))
            
    evaluate(sess,X,Y)
    
    coord.request_stop()
    coord.join(threads)
    sess.close()

loss :  [0.38171607]
loss :  [-8.257179]
loss :  [-23.042315]
loss :  [-21.998028]
loss :  [-25.212051]
loss :  [-41.931725]
loss :  [-77.137794]
loss :  [-43.682335]
loss :  [-51.797993]
loss :  [-94.33719]
loss :  [-96.6888]
loss :  [-86.28653]
loss :  [-103.04562]
loss :  [-76.58591]
loss :  [-121.12936]
loss :  [-121.10418]
loss :  [-149.81291]
loss :  [-87.44158]
loss :  [-154.61986]
loss :  [-168.48415]
loss :  [-156.41003]
loss :  [-136.92178]
loss :  [-181.62753]
loss :  [-235.26535]
loss :  [-178.49307]
loss :  [-175.19508]
loss :  [-210.30003]
loss :  [-261.21332]
loss :  [-223.45883]
loss :  [-154.97787]
loss :  [-222.91188]
loss :  [-337.79907]
loss :  [-276.92426]
loss :  [-264.1828]
loss :  [-271.0176]
loss :  [-352.429]
loss :  [-457.20483]
loss :  [-296.40628]
loss :  [-253.77727]
loss :  [-320.42615]
loss :  [-416.29962]
loss :  [-398.7914]
loss :  [-266.67416]
loss :  [-374.18677]
loss :  [-368.7137]
loss :  [-425.23776]
loss :  [-403.92673]
loss :  [-253.10945]
loss 